In [5]:
import requests
import pandas as pd
import numpy as np
from datetime import date
import xlrd

# here you have to enter your actual API key from SimFin
api_key = "khJmsEQpgG1bK7Xl1vgXwyv9SCbrsBSP"

# list of tickers we want to get data for
tickers = ["AAPL",
            "NVDA",
            "WMT",
            "MSFT",
            "GOOG",
            "AMZN",
            "FB",
            "BRK.B",
            "JPM",
            "V",
            "JNJ",
            "XOM",
            "BAC",
            "PG",
            "T",
            "MA",
            "HD",
            "INTC",
            "DIS",
            "VZ",
            "UNH"]

# first: find SimFin IDs
sim_ids = []
for ticker in tickers:

    request_url = f'https://simfin.com/api/v1/info/find-id/ticker/{ticker}?api-key={api_key}'
    content = requests.get(request_url)
    data = content.json()

    if "error" in data or len(data) < 1:
        sim_ids.append(None)
    else:
        sim_ids.append(data[0]['simId'])

print(sim_ids)

# define time periods for financial statement data
statement_type = "pl"
time_periods = ["Q1", "Q2", "Q3", "Q4"]
year_start = 2015
year_end = 2019

# set up the XLSX writer
writer = pd.ExcelWriter("simfin_data_pl.xlsx", engine='xlsxwriter')

data = {}
# get data for each ticker/id
for idx, sim_id in enumerate(sim_ids):
    d = data[tickers[idx]] = {"Line Item": []}
    if sim_id is not None:
        for year in range(year_start, year_end + 1):
            for time_period in time_periods:

                # make time period identifier
                period_identifier = time_period + "-" + str(year)

                if period_identifier not in d:
                    d[period_identifier] = []

                request_url = f'https://simfin.com/api/v1/companies/id/{sim_id}/statements/standardised?stype={statement_type}&fyear={year}&ptype={time_period}&api-key={api_key}'

                content = requests.get(request_url)
                statement_data = content.json()

                # collect line item names once, they are the same for all companies with the standardised data
                if len(d['Line Item']) == 0 and 'values' in statement_data:
                    d['Line Item'] = [x['standardisedName'] for x in statement_data['values']]

                if 'values' in statement_data:
                    for item in statement_data['values']:
                        d[period_identifier].append(item['valueChosen'])
                else:
                    # no data found for time period
                    d[period_identifier] = [None for _ in d['Line Item']]

        # fix the periods where no values were available
        len_target = len(d['Line Item'])
        if len_target > 0:
            for k, v in d.items():
                if len(v) != len_target:
                    d[k] = [None for _ in d['Line Item']]

    # convert to pandas dataframe
    df = pd.DataFrame(data=d)
    df["symbol"]=tickers[idx]
    # save in the XLSX file configured earlier
    df.to_excel(writer, sheet_name=tickers[idx])#,index=False)#get rid of the unnamed ON THE CONCAT

writer.save()
writer.close()

[111052, 172199, 239962, 59265, 18, 62747, 121021, None, 243510, 107326, 89661, 121214, 71941, 133209, 66790, 331628, 261673, 85652, 261356, 101219, 367714]


PermissionError: [Errno 13] Permission denied: 'simfin_data_pl.xlsx'

In [ ]:
loc=("../project_22/simfin_data_pl.xlsx")
wb=xlrd.open_workbook(loc) #basically a pd.csv

In [ ]:
df.to_csv(r'C:\Users\roel Cantu\Desktop\project_2.csv')
df.head(2)

In [ ]:
combine_df=pd.concat(pd.read_excel("simfin_data_pl.xlsx",sheet_name=None), ignore_index=True, sort=True)
combine_df.head(2)

In [ ]:
df.set_index("Line Item", inplace=True)

In [6]:
AAPL_df=pd.read_excel("simfin_data_pl.xlsx", sheet_name="AAPL")
AAPL_df.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
AAPL_df.drop(["a"], axis=1, inplace=True)
AAPL_df=AAPL_df.dropna(axis='columns', how='all').dropna(axis="rows", how="any")
AAPL_df.set_index("Line Item",inplace=True)
AAPL_df.drop(columns= "symbol",inplace=True)
AAPL_df=AAPL_df.T
AAPL_df.head(2)


Line Item,Revenue,Cost of revenue,Gross Profit,Operating Expenses,"Selling, General & Administrative",Research & Development,Operating Income (Loss),Non-Operating Income (Loss),Other Non-Operating Income (Loss),"Pretax Income (Loss), Adjusted",Pretax Income (Loss),"Income Tax (Expense) Benefit, net",Income (Loss) from Continuing Operations,Income (Loss) Including Minority Interest,Net Income,Net Income Available to Common Shareholders
Q1-2015,7.459900e+10,-4.485800e+10,2.974100e+10,-5.495000e+09,-3.600000e+09,-1.895000e+09,2.424600e+10,170000000.0,170000000.0,2.441600e+10,2.441600e+10,-6.392000e+09,1.802400e+10,1.802400e+10,1.802400e+10,1.802400e+10
Q2-2015,5.801000e+10,-3.435400e+10,2.365600e+10,-5.378000e+09,-3.460000e+09,-1.918000e+09,1.827800e+10,286000000.0,286000000.0,1.856400e+10,1.856400e+10,-4.995000e+09,1.356900e+10,1.356900e+10,1.356900e+10,1.356900e+10


In [ ]:
NVDA_df=pd.read_excel("simfin_data_pl.xlsx", sheet_name="NVDA")
NVDA_df.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
NVDA_df.drop(["a"], axis=1, inplace=True)
NVDA_df=NVDA_df.dropna(axis='columns', how='all').dropna(axis="rows", how="any")
NVDA_df.set_index("Line Item",inplace=True)
NVDA_df.drop(columns= "symbol",inplace=True)
NVDA_df=NVDA_df.T
NVDA_df.head(2)

In [ ]:
WMT_df=pd.read_excel("simfin_data_pl.xlsx", sheet_name="WMT")
WMT_df.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
WMT_df.drop(["a"], axis=1, inplace=True)
WMT_df=WMT_df.dropna(axis='columns', how='all').dropna(axis="rows", how="any")
WMT_df.set_index("Line Item",inplace=True)
WMT_df.drop(columns= "symbol",inplace=True)
WMT_df=WMT_df.T
WMT_df.head(2)

In [13]:
MSFT_df=pd.read_excel("simfin_data_pl.xlsx", sheet_name="MSFT")
MSFT_df.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
MSFT_df.drop(["a"], axis=1, inplace=True)
MSFT_df=MSFT_df.dropna(axis='columns', how='all').dropna(axis="rows", how="any")
MSFT_df.set_index("Line Item",inplace=True)
MSFT_df.drop(columns= "symbol",inplace=True)
MSFT_df=MSFT_df.T
MSFT_df.head(2)

Line Item,Revenue,Cost of revenue,Gross Profit,Operating Expenses,"Selling, General & Administrative",Selling & Marketing,General & Administrative,Research & Development,Operating Income (Loss),Non-Operating Income (Loss),Other Non-Operating Income (Loss),"Pretax Income (Loss), Adjusted",Pretax Income (Loss),"Income Tax (Expense) Benefit, net",Income (Loss) from Continuing Operations,Income (Loss) Including Minority Interest,Net Income,Net Income Available to Common Shareholders
Q1-2015,2.320100e+10,-8.273000e+09,1.492800e+10,-7.944000e+09,-4.879000e+09,-3.728000e+09,-1.151000e+09,-3.065000e+09,6.984000e+09,52000000.0,52000000.0,7.036000e+09,5.896000e+09,-1.356000e+09,4.540000e+09,4.540000e+09,4.540000e+09,4.540000e+09
Q2-2015,2.647000e+10,-1.013600e+10,1.633400e+10,-8.315000e+09,-5.412000e+09,-4.315000e+09,-1.097000e+09,-2.903000e+09,8.019000e+09,74000000.0,74000000.0,8.093000e+09,7.850000e+09,-1.987000e+09,5.863000e+09,5.863000e+09,5.863000e+09,5.863000e+09


In [16]:

GOOG_df=pd.read_excel("simfin_data_pl.xlsx", sheet_name="GOOG")
GOOG_df.rename({"Unnamed: 0":"Line Item"}, axis="columns", inplace=True)
GOOG_df=GOOG_df.dropna(axis='columns', how="all").dropna(axis="rows", how="any")
GOOG_df.set_index("Line Item",inplace=True)
#GOOG_df = df.iloc[1:]
#GOOG_df.drop(columns= "symbol",inplace=True)
GOOG_df=GOOG_df.T
GOOG_df

Line Item,Revenue,Cost of revenue,Gross Profit,Operating Expenses,Operating Income (Loss),Non-Operating Income (Loss),"Pretax Income (Loss), Adjusted",Pretax Income (Loss),"Income Tax (Expense) Benefit, net",Net Income,Net Income Available to Common Shareholders
Q1 '15,17258.0,-6356.0,10902.0,-6455.0,4447.0,157.0,4604.0,4604.0,-1089.0,3515.0,3515.0
Q2 '15,17727.0,-6583.0,11144.0,-6319.0,4825.0,131.0,4956.0,4956.0,-1025.0,3931.0,3409.0
Q3 '15,18675.0,-7037.0,11638.0,-6930.0,4708.0,183.0,4891.0,4891.0,-912.0,3979.0,3979.0
Q4 '15,21329.0,-8188.0,13141.0,-7761.0,5380.0,-180.0,5200.0,5200.0,-277.0,4923.0,4923.0
Q1 '16,20257.0,-7648.0,12609.0,-7267.0,5342.0,-213.0,5129.0,5129.0,-922.0,4207.0,4207.0
Q2 '16,21500.0,-8130.0,13370.0,-7402.0,5968.0,151.0,6119.0,6119.0,-1242.0,4877.0,4877.0
Q3 '16,22451.0,-8699.0,13752.0,-7985.0,5767.0,278.0,6045.0,6045.0,-984.0,5061.0,5061.0
Q4 '16,26064.0,-10661.0,15403.0,-8764.0,6639.0,218.0,6857.0,6857.0,-1524.0,5333.0,5333.0
Q1 '17,24750.0,-9795.0,14955.0,-8387.0,6568.0,251.0,6819.0,6819.0,-1393.0,5426.0,5426.0
Q2 '17,26010.0,-10373.0,15637.0,-8769.0,6868.0,245.0,7113.0,4377.0,-853.0,3524.0,3524.0


In [ ]:
AMZN_df=pd.read_excel("simfin_data_pl.xlsx", sheet_name="AMZN")
AMZN_df.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
AMZN_df.drop(["a"], axis=1, inplace=True)
AMZN_df=AMZN_df.dropna(axis='columns', how='all').dropna(axis="rows", how="any")
AMZN_df.set_index("Line Item",inplace=True)
AMZN_df.drop(columns= "symbol",inplace=True)
AMZN_df=AMZN_df.T
AMZN_df.head(2)

In [ ]:
FB_df=pd.read_excel("simfin_data_pl.xlsx", sheet_name="FB")
FB_df.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
FB_df.drop(["a"], axis=1, inplace=True)
FB_df=FB_df.dropna(axis='columns', how='all').dropna(axis="rows", how="any")
FB_df.set_index("Line Item",inplace=True)
FB_df.drop(columns= "symbol",inplace=True)
FB_df=FB_df.T
FB_df.head(2)

In [ ]:
JPM_df=pd.read_excel("simfin_data_pl.xlsx", sheet_name="JPM")
JPM_df.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
JPM_df.drop(["a"], axis=1, inplace=True)
JPM_df=JPM_df.dropna(axis='columns', how='all').dropna(axis="rows", how="any")
JPM_df.set_index("Line Item",inplace=True)
JPM_df.drop(columns= "symbol",inplace=True)
JPM_df=JPM_df.T
JPM_df.head(2)

In [ ]:
V_df=pd.read_excel("simfin_data_pl.xlsx", sheet_name="V")
V_df.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
V_df.drop(["a"], axis=1, inplace=True)
V_df=V_df.dropna(axis='columns', how='all').dropna(axis="rows", how="any")
V_df.set_index("Line Item",inplace=True)
V_df.drop(columns= "symbol",inplace=True)
V_df=V_df.T
V_df.head(2)

In [ ]:
JNJ_df=pd.read_excel("simfin_data_pl.xlsx", sheet_name="JNJ")
JNJ_df.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
JNJ_df.drop(["a"], axis=1, inplace=True)
JNJ_df=JNJ_df.dropna(axis='columns', how='all').dropna(axis="rows", how="any")
JNJ_df.set_index("Line Item",inplace=True)
JNJ_df.drop(columns= "symbol",inplace=True)
JNJ_df=JNJ_df.T
JNJ_df.head(2)

In [ ]:
XOM_df=pd.read_excel("simfin_data_pl.xlsx", sheet_name="XOM")
XOM_df.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
XOM_df.drop(["a"], axis=1, inplace=True)
XOM_df=XOM_df.dropna(axis='columns', how='all').dropna(axis="rows", how="any")
XOM_df.set_index("Line Item",inplace=True)
XOM_df.drop(columns= "symbol",inplace=True)
XOM_df=XOM_df.T
XOM_df.head(2)

In [ ]:
BAC_df=pd.read_excel("simfin_data_pl.xlsx", sheet_name="BAC")
BAC_df.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
BAC_df.drop(["a"], axis=1, inplace=True)
BAC_df=BAC_df.dropna(axis='columns', how='all').dropna(axis="rows", how="any")
BAC_df.set_index("Line Item",inplace=True)
BAC_df.drop(columns= "symbol",inplace=True)
BAC_df=BAC_df.T
BAC_df.head(2)

In [ ]:
PG_df=pd.read_excel("simfin_data_pl.xlsx", sheet_name="PG")
PG_df.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
PG_df.drop(["a"], axis=1, inplace=True)
PG_df=PG_df.dropna(axis='columns', how='all').dropna(axis="rows", how="any")
PG_df.set_index("Line Item",inplace=True)
PG_df.drop(columns= "symbol",inplace=True)
PG_df=PG_df.T
PG_df.head(2)

In [ ]:
T_df=pd.read_excel("simfin_data_pl.xlsx", sheet_name="T")
T_df.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
T_df.drop(["a"], axis=1, inplace=True)
T_df=T_df.dropna(axis='columns', how='all').dropna(axis="rows", how="any")
T_df.set_index("Line Item",inplace=True)
T_df.drop(columns= "symbol",inplace=True)
T_df=T_df.T
T_df.head(2)

In [ ]:
HD_df=pd.read_excel("simfin_data_pl.xlsx", sheet_name="HD")
HD_df.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
HD_df.drop(["a"], axis=1, inplace=True)
HD_df=HD_df.dropna(axis='columns', how='all').dropna(axis="rows", how="any")
HD_df.set_index("Line Item",inplace=True)
HD_df.drop(columns= "symbol",inplace=True)
HD_df=HD_df.T
HD_df.head(2)

In [ ]:
MA_df=pd.read_excel("simfin_data_pl.xlsx", sheet_name="MA")
MA_df.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
MA_df.drop(["a"], axis=1, inplace=True)
MA_df=MA_df.dropna(axis='columns', how='all').dropna(axis="rows", how="any")
MA_df.set_index("Line Item",inplace=True)
MA_df.drop(columns= "symbol",inplace=True)
MA_df=MA_df.T
MA_df.head(2)

In [ ]:
INTC_df=pd.read_excel("simfin_data_pl.xlsx", sheet_name="INTC")
INTC_df.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
INTC_df.drop(["a"], axis=1, inplace=True)
INTC_df=INTC_df.dropna(axis='columns', how='all').dropna(axis="rows", how="any")
INTC_df.set_index("Line Item",inplace=True)
INTC_df.drop(columns= "symbol",inplace=True)
INTC_df=INTC_df.T
INTC_df.head(2)

In [ ]:
DIS_df=pd.read_excel("simfin_data_pl.xlsx", sheet_name="DIS")
DIS_df.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
DIS_df.drop(["a"], axis=1, inplace=True)
DIS_df=DIS_df.dropna(axis='columns', how='all').dropna(axis="rows", how="any")
DIS_df.set_index("Line Item",inplace=True)
DIS_df.drop(columns= "symbol",inplace=True)
DIS_df=DIS_df.T
DIS_df.head(2)

In [ ]:
VZ_df=pd.read_excel("simfin_data_pl.xlsx", sheet_name="VZ")
VZ_df.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
VZ_df.drop(["a"], axis=1, inplace=True)
VZ_df=VZ_df.dropna(axis='columns', how='all').dropna(axis="rows", how="any")
VZ_df.set_index("Line Item",inplace=True)
VZ_df.drop(columns= "symbol",inplace=True)
VZ_df=VZ_df.T
VZ_df.head(2)

In [ ]:
UNH_df=pd.read_excel("simfin_data_pl.xlsx", sheet_name="UNH")
UNH_df.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
UNH_df.drop(["a"], axis=1, inplace=True)
UNH_df=UNH_df.dropna(axis='columns', how='all').dropna(axis="rows", how="any")
UNH_df.set_index("Line Item",inplace=True)
UNH_df.drop(columns= "symbol",inplace=True)
UNH_df=UNH_df.T
UNH_df.head(2)

In [ ]:
#Liquidity Ratios
Current ratio = Current assets / Current liabilities
Acid-test ratio = Current assets – Inventories / Current liabilities
Cash ratio = Cash and Cash equivalents / Current Liabilities
Operating cash flow ratio = Operating cash flow / Current liabilities
#Leverage Financial Ratios
Debt ratio = Total liabilities / Total assets
Debt to equity ratio = Total liabilities / Shareholder’s equity
Interest coverage ratio = Operating income / Interest expenses
Debt service coverage ratio = Operating income / Total debt service
#Efficiency Ratios
Asset turnover ratio = Net sales / Total assets
Inventory turnover ratio = Cost of goods sold / Average inventory
Receivables turnover ratio = Net credit sales / Average accounts receivable
#profitability ratios
Gross margin ratio = Gross Profit / Net sales
Operating margin ratio = Operating income / Net sales
Return on assets ratio = Net income / Total assets
# no data for market value ratios